In [3]:
import sklearn.model_selection
from sklearn.model_selection import permutation_test_score
# warning settings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data management
import pandas as pd
import numpy as np
import pickle

# Plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# HDDM
import hddm
import scipy.stats
# from scipy.stats import permutation_test
import random


ModuleNotFoundError: No module named 'sklearn'

In [2]:
df_orig = pd.read_csv('/data2/victoria/brain_data/brain_behav_late_learn.csv')
# df = df_orig.loc[:,['sbj','correct','rt','trial','run','type','learner','roi1']]
# df.columns = ('subj_idx','response','rt','trial','run','type','learner','roi1')
df = df_orig
df.rt = df.rt/1000
df = hddm.utils.flip_errors(df)
df = df.dropna()
len(df)

1604

split df into behavioral and roi two dfs, and only shuffle the behave df
later concat the two dfs again

In [5]:

roi_df = df.iloc[:,-108:]
behave_df =df.iloc[:,1:17]


loop 1000 times for each roi
run all the models after each shuffle

In [6]:
roi_ls = [41]
for i in range(0,1):
    num_row = 0
    behav_shu = pd.DataFrame()
    for p in range(100,143):
        if p in behave_df['sbj'].values:
            subj_df = behave_df[behave_df['sbj']== p] #identify each subject and put into a smaller df
            nrow= len(subj_df)
            count_5 = subj_df['run'].value_counts()[5]
            count_6 = subj_df['run'].value_counts()[6]
            run_5 = [5] * count_5
            run_6 = [6] * count_6
            new_run = run_5 + run_6
            rand_5 = random.sample(range(1,count_5+1),count_5)
            rand_6 = random.sample(range(1,count_6+1),count_6)
            rand_int = rand_5 + rand_6
            subj_df['runtrial'] = rand_int
            subj_df['run'] = new_run
            subj_sorted = subj_df.sort_values(by=['run', 'runtrial'])
            # subj_sorted.to_csv(f'{p}_sorted.csv')
            behav_shu = pd.concat([behav_shu,subj_sorted], axis=0) # concat dfs vertically
    # df_shu = subj_df.sort_values(by=['run', 'runtrial'])
    # df_noind = behav_shu.iloc[:,1:]
    behav_shu = behav_shu.reset_index(drop=True)
    roi_df = roi_df.reset_index(drop=True)
    df_shu = pd.concat([behav_shu,roi_df], axis=1)
    #behav_shu.to_csv(f'behav_shu{i}.csv')
    #df_shu.to_csv(f'whole_shuffled{i}.csv')
   
    

/data/software/miniconda3/envs/hddm0.9/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/data/software/miniconda3/envs/hddm0.9/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# roi_ls = [31]
for roi in roi_ls:
        m_reg = hddm.HDDMRegressor(df_shu,f"a ~ roi{roi} * C(type,Treatment('prototype'))",
                                p_outlier=0.05)  
        m_reg.sample(50, burn=5)
        m_reg.print_stats()
        print(f" roi{roi} DIC value: {m_reg.dic}")
        

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------102%-----------------] 51 of 50 complete in 32.7 sec                                                               mean         std       2.5q        25q        50q        75q      97.5q      mc err
v                                                         0.0515884   0.0224226  0.0129491  0.0327559  0.0582597  0.0685959  0.0826121  0.00334256
t                                                          0.362283  0.00729788   0.344129   0.359933    0.36355   0.367119   0.372746   0.0010879
a_Intercept                                                 1.71332   0.0511258    1.65881    1.67917    1.70798    1.72492    1.85895  0.00762139
a_C(type, Treatment('prototype'))[T.exception]             0.122624   0.0826193 -0.0812538  0.0870424    0.12805   0.169368   0.268258   0.0123162
a_C(type, Treatment('prototype'))[T.rule follower]         0.130305   0.0785184 -0.0841472   0.092647   0.134901   0.176086    0.

In [15]:
stats = m_reg.gen_stats()

In [19]:
print(m_reg.dic)

4587.792264986136


In [ ]:
for roi in roi_ls:
        m_reg = hddm.HDDMRegressor(df_shu,f"v ~ roi{roi} * C(type,Treatment('prototype'))",
                                p_outlier=0.05)  
        m_reg.sample(2000, burn=1000)
        # m_reg.print_stats()
        